In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [0]:
cd /content/drive/My Drive/Deep_Learning_Codes/CNN_model_restore

In [0]:
import tensorflow as tf
import numpy as np
import h5py
from keras.datasets import cifar10

In [0]:
def load_data(clases, caracteristicas):
  (X_train, Y_tr), (X_test, Y_te) = cifar10.load_data()
  
  Train_samples=Y_tr.size
  Y_train=np.zeros((Train_samples,clases))
  for i in range(Train_samples):
      Y_train[i,Y_tr[i]]=1    

  Test_samples=Y_te.size
  Y_test=np.zeros((Test_samples,clases))
  for i in range(Test_samples):
      Y_test[i,Y_te[i]]=1
  return (X_train/255,X_test/255,Y_train,Y_test)

In [0]:
def create_placeholders(n_input,n_classes):
    X = tf.placeholder("float", [None, n_input[0],n_input[1],n_input[2]])
    Y = tf.placeholder("float", [None, n_classes])
    return (X,Y)

In [0]:
def initialize_parameters(n_input, K_size_1, K_size_2, K_size_3, N_filters_1, N_filters_2, N_filters_3):
    W1 = tf.get_variable("W1", [K_size_1, K_size_1, n_input[2], N_filters_1], initializer = 
    							tf.contrib.layers.xavier_initializer(seed = 0))
    W2 = tf.get_variable("W2", [K_size_2, K_size_2, N_filters_1, N_filters_2], initializer = 
    							tf.contrib.layers.xavier_initializer(seed = 0))
    W3 = tf.get_variable("W3", [K_size_3, K_size_3, N_filters_2, N_filters_3], initializer = 
    							tf.contrib.layers.xavier_initializer(seed = 0))
    parameters = {"W1": W1,
                  "W2": W2,
                  "W3": W3,}
    
    return parameters

In [0]:
def forward_propagation(X, parameters, n_classes):
    W1 = parameters['W1']
    W2 = parameters['W2']
    W3 = parameters['W3']
    #Definicion capas convolucionales
    A1 = tf.nn.relu(tf.nn.conv2d(X, W1, strides = [1,1,1,1], padding = 'SAME'))
    P1 = tf.nn.max_pool(A1, ksize = [1,8,8,1], strides = [1,8,8,1], padding = 'SAME')
    A2 = tf.nn.relu(tf.nn.conv2d(P1,W2, strides = [1,1,1,1], padding = 'SAME'))
    P2 = tf.nn.max_pool(A2, ksize = [1,8,8,1], strides = [1,8,8,1], padding = 'SAME')
    A3 = tf.nn.relu(tf.nn.conv2d(P2,W3, strides = [1,1,1,1], padding = 'SAME'))
    F1 = tf.contrib.layers.flatten(A3)
    # Definición del perceptrón multicapa
    F2 = tf.contrib.layers.fully_connected(F1, 128, None)
    out_layer = tf.contrib.layers.fully_connected(F2, n_classes, None)
    return out_layer

In [0]:
def compute_cost(logits, Y):
    loss_op=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
    return loss_op

In [0]:
def train(training_epochs, batch_size, display_step):
    #Objeto para el almacenamiento de los pesos
    saver = tf.train.Saver()
    with tf.Session() as sess:
        sess.run(init)
        # Ciclo en el que se repetirá el proceso de entrenamiento
        for epoch in range(training_epochs):
            avg_cost = 0.
            #obtiene el numero de grupos en que queda dividida la base de datos
            total_batch = int(Y_train.shape[0]/batch_size) 
            # ciclo para entrenar con cada grupo de datos
            for i in range(total_batch-1):
                batch_x= X_train[i*batch_size:(i+1)*batch_size]
                batch_y= Y_train[i*batch_size:(i+1)*batch_size]
                # Correr la funcion de perdida y la operacion de optimización 
                # con la respectiva alimentación del placeholder
                _,c =sess.run([train_op, cost],feed_dict={X:batch_x,Y:batch_y})
                # Promedio de resultados de la funcion de pérdida
                avg_cost += c / total_batch
            # Mostrar el resultado del entrenamiento por grupos
            if epoch % display_step == 0:
                print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
        print("Optimization Finished!")
        saver.save(sess, "saved_model/model.ckpt")
        # Calculo de presición
        traa=0.
        for i in range(total_batch-1):
            batch_x= X_train[i*batch_size:(i+1)*batch_size]
            batch_y= Y_train[i*batch_size:(i+1)*batch_size]
            acc=sess.run(accuracy,feed_dict={X:batch_x,Y:batch_y})
            traa += acc / total_batch
        teaa=0.
        total_batch = int(Y_test.shape[0]/batch_size) 
        for i in range(total_batch-1):
            batch_x= X_test[i*batch_size:(i+1)*batch_size]
            batch_y= Y_test[i*batch_size:(i+1)*batch_size]
            acc=sess.run(accuracy,feed_dict={X: batch_x, Y: batch_y})
            teaa += acc / total_batch
        print("Train_Accuracy:", acc)
        print("Test_Accuracy:", acc)

In [0]:
tf.reset_default_graph() #Linea para eliminar el grafo que se encuentra en memoria

# Parametros de entrenamiento
learning_rate = 0.001
training_epochs = 5
batch_size = 256
display_step = 1

# Parámetros para la contrucción del modelo
K_size_1 = 5 # tamaño del Kernel (Ej:5*5)
K_size_2 = 5 
K_size_3 = 3 
N_filters_1 = 6
N_filters_2 = 12
N_filters_3 = 24
n_input = (32,32,3) # Caracteristicas de entrada (ej:28*28 pixeles (mnist))
n_classes = 10 # Total de clases en la base de datos

#Cargar la base de datos para entrenar
X_train,X_test,Y_train,Y_test=load_data(n_classes, n_input)

# CONSTRUCCION DEL GRAFO
################################################################################
# Crear placeholders
X,Y = create_placeholders(n_input,n_classes)
# Definir parámetros
parameters = initialize_parameters(n_input, K_size_1, K_size_2, K_size_3, N_filters_1, N_filters_2, N_filters_3)

# Declarar la operación que aplica el MLP usando la información de entrada
logits = forward_propagation(X, parameters, n_classes)

# Declarar las operaciónes que establecen la funcion de costo y optimización
cost = compute_cost(logits, Y)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(cost)

# Declarar operaciones para el calculo de la precisión 
pred_onehot = tf.nn.softmax(logits)  # aplicando softmax al modelo
prediccion= tf.argmax(pred_onehot, 1)
correct_prediction = tf.equal(prediccion, tf.argmax(Y, 1)) #comparando cada salida de la red con las etiquetas
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))# (prediciones correctas)/(totales)


# inicializador de variables
init = tf.global_variables_initializer()
################################################################################

# Inicio de sesion para la ejecusión del Grafo
train(training_epochs, batch_size, display_step)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch: 0001 cost=2.021045747
Epoch: 0002 cost=1.730590882
Epoch: 0003 cost=1.653063681
Epoch: 0004 cost=1.599383724
Epoch: 0005 cost=1.562964313
Optimization Finished!


In [0]:
# CODIGO PARA USAR EL MODELO PARA LA INFERENCIA
def model_predict(data_test):
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, "saved_model/model.ckpt")        
        print("Model restored.")
        pred=sess.run([prediccion], feed_dict={X: data_test})
        return pred

for indice in range (Y_test.shape[0]):
    pred=model_predict([X_test[indice]])
    print("\n\nreal:", np.argmax(Y_test[indice]))
    print("prediccion:", pred)